<a href="https://colab.research.google.com/github/heet25itachi/TIMEPASS_BUDDY/blob/main/Microsoft%20Drive%20Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
    from IPython.display import HTML

    html_content = "<h2>Dynamic HTML from Python</h2><p>The current time is: "
    # Add a timestamp or other dynamic content
    import datetime
    html_content += str(datetime.datetime.now()) + "</p>"

    display(HTML(html_content))

In [7]:
%%writefile drive_optimizer.cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <algorithm>
#include <chrono>
#include <thread>
#include <limits>

using namespace std;

// --- Constants ---
const int TOTAL_BLOCKS = 2500;
const int DISK_WIDTH = 50;
const int BLOCK_SIZE = 4; // Simulated block size in MB
const float USED_RATE = 0.70f;
const float FRAGMENTATION_RATE = 0.40f;

// --- Enumerations and Structs ---
enum BlockType { FREE, CONTIGUOUS, FRAGMENTED };
struct DiskBlock {
    int id;
    int file_id;
    BlockType type;
};

// --- Utility Functions ---
void clear_screen() {
    cout << "\033[2J\033[1;1H";
}
void sleep_ms(int ms) {
    this_thread::sleep_for(chrono::milliseconds(ms));
}

// --- Core Logic (Initialization) ---
void initialize_disk(vector<DiskBlock>& disk_array) {
    disk_array.clear();
    srand(static_cast<unsigned int>(time(0)));

    int used_blocks_count = static_cast<int>(TOTAL_BLOCKS * USED_RATE);
    int fragmented_blocks_count = static_cast<int>(used_blocks_count * FRAGMENTATION_RATE);

    int current_id = 0;
    int current_file_id = 1;

    for (int i = 0; i < used_blocks_count; ++i) {
        DiskBlock block = {current_id++, current_file_id, CONTIGUOUS};

        if (i < fragmented_blocks_count) {
            block.type = FRAGMENTED;
        }

        if (i % 10 == 0) {
            current_file_id++;
        }
        disk_array.push_back(block);
    }

    for (int i = used_blocks_count; i < TOTAL_BLOCKS; ++i) {
        disk_array.push_back({current_id++, -1, FREE});
    }

    random_shuffle(disk_array.begin(), disk_array.end());

    for(int i = 0; i < TOTAL_BLOCKS; ++i) {
        disk_array[i].id = i;
    }
}

// --- Core Logic (Drawing) ---
void draw_disk(const vector<DiskBlock>& disk_array, const string& title) {
    clear_screen();
    cout << "--- " << title << " ---\n\n";
    cout << "Legend: \033[31mF\033[0m=Fragmented | \033[32mC\033[0m=Contiguous | \033[37m.\033[0m=Free Space\n";
    cout << "Total Blocks: " << TOTAL_BLOCKS << " (" << (TOTAL_BLOCKS * BLOCK_SIZE / 1024.0) << " GB Sim.)\n";
    cout << "Disk Map:\n";

    for (int i = 0; i < TOTAL_BLOCKS; ++i) {
        const DiskBlock& block = disk_array[i];

        char symbol = '.';
        string color_code = "\033[37m";

        if (block.type == FRAGMENTED) {
            symbol = 'F';
            color_code = "\033[31m";
        } else if (block.type == CONTIGUOUS) {
            symbol = 'C';
            color_code = "\033[32m";
        }

        cout << color_code << symbol << "\033[0m";

        if ((i + 1) % DISK_WIDTH == 0) {
            cout << "\n";
        }
    }
    cout << "\n";
}

// --- Core Logic (Analysis) ---
float analyze_disk(const vector<DiskBlock>& disk_array) {
    int fragmented_count = 0;
    int used_count = 0;

    for (const auto& block : disk_array) {
        if (block.type != FREE) {
            used_count++;
            if (block.type == FRAGMENTED) {
                fragmented_count++;
            }
        }
    }

    if (used_count == 0) return 0.0f;
    return (static_cast<float>(fragmented_count) / used_count) * 100.0f;
}

// --- Core Logic (Optimization) ---
void optimize_disk(vector<DiskBlock>& disk_array) {
    int free_cursor = 0;
    int blocks_moved = 0;
    const int SLEEP_TIME = 10;

    while (free_cursor < TOTAL_BLOCKS && disk_array[free_cursor].type != FREE) {
        free_cursor++;
    }

    if (free_cursor >= TOTAL_BLOCKS) {
        draw_disk(disk_array, "Optimization Skipped: Disk is already full and optimized.");
        sleep_ms(2000);
        return;
    }

    for (int i = free_cursor; i < TOTAL_BLOCKS; ++i) {
        if (disk_array[i].type != FREE) {

            DiskBlock data_block = disk_array[i];

            disk_array[free_cursor].file_id = data_block.file_id;
            disk_array[free_cursor].type = CONTIGUOUS;

            disk_array[i].file_id = -1;
            disk_array[i].type = FREE;

            free_cursor++;
            blocks_moved++;

            if (blocks_moved % 5 == 0) {
                draw_disk(disk_array, "Optimizing... Blocks Moved: " + to_string(blocks_moved));
                sleep_ms(SLEEP_TIME);
            }
        }
    }

    draw_disk(disk_array, "Optimization Complete! Blocks moved: " + to_string(blocks_moved));
    sleep_ms(2000);
}


// --- Main Program Loop ---
int main() {
    vector<DiskBlock> disk_c;
    char choice;

    initialize_disk(disk_c);

    do {
        draw_disk(disk_c, "C: Drive Status");
        float frag_percent = analyze_disk(disk_c);

        cout << "\n------------------------------------------------\n";
        cout << "Current Fragmentation: " << frag_percent << "%\n";
        cout << "------------------------------------------------\n";
        cout << "(A) Analyze | (O) Optimize | (R) Re-fragment | (Q) Quit\n";
        cout << "Enter command: ";

        if (!(cin >> choice)) {
            cin.clear();
            cin.ignore(numeric_limits<streamsize>::max(), '\n');
            continue;
        }

        choice = toupper(choice);

        if (choice == 'A') {
            draw_disk(disk_c, "Analyzing Disk...");
            sleep_ms(1000);
            cout << "\n\nAnalysis finished. Fragmentation is: " << frag_percent << "%\n";
            sleep_ms(1500);

        } else if (choice == 'O') {
            if (frag_percent > 1.0f) {
                optimize_disk(disk_c);
            } else {
                draw_disk(disk_c, "Optimization not necessary (<1% fragmentation).");
                sleep_ms(1500);
            }
        } else if (choice == 'R') {
            initialize_disk(disk_c);
            draw_disk(disk_c, "Disk Re-Fragmented (Simulated Heavy Usage)");
            sleep_ms(1500);
        } else if (choice != 'Q') {
            cout << "Invalid choice. Please try again.\n";
            sleep_ms(1000);
        }

    } while (choice != 'Q');

    clear_screen();
    cout << "Drive Optimizer Simulator Exited.\n";
    return 0;
}

### **Colab Cell 2: Compile**

This cell uses the g++ compiler to create an executable file named `drive_optimizer`.

```bash
!g++ drive_optimizer.cpp -o drive_optimizer -std=c++17

### **Colab Cell 3: Execute**

This cell runs the compiled program. You will interact with the simulator directly in the output area below this cell.

```bash
!./drive_optimizer

Writing drive_optimizer.cpp


In [9]:
%%writefile drive_optimizer.cpp
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>
#include <algorithm>
#include <chrono>
#include <thread>
#include <limits> // Required for numeric_limits

using namespace std;

// --- Constants ---
const int TOTAL_BLOCKS = 2500;
const int DISK_WIDTH = 50;
const int BLOCK_SIZE = 4; // Simulated block size in MB
const float USED_RATE = 0.70f;
const float FRAGMENTATION_RATE = 0.40f;

// --- Enumerations and Structs ---

// Defines the state of a disk block
enum BlockType { FREE, CONTIGUOUS, FRAGMENTED };

// Represents a single block on the disk
struct DiskBlock {
    int id;
    int file_id; // Identifies which file the fragment belongs to
    BlockType type;
};

// --- Utility Functions ---

void clear_screen() {
    // Colab/Terminal compatible screen clear using ANSI escape sequences
    cout << "\033[2J\033[1;1H";
}

void sleep_ms(int ms) {
    // Pause execution for visualization
    this_thread::sleep_for(chrono::milliseconds(ms));
}

// --- Core Logic ---

/**
 * @brief Initializes the disk with a mix of free, contiguous, and fragmented blocks.
 * @param disk_array Reference to the vector representing the disk.
 */
void initialize_disk(vector<DiskBlock>& disk_array) {
    disk_array.clear();
    // Use the current time to seed the random number generator
    srand(static_cast<unsigned int>(time(0)));

    int used_blocks_count = static_cast<int>(TOTAL_BLOCKS * USED_RATE);
    int fragmented_blocks_count = static_cast<int>(used_blocks_count * FRAGMENTATION_RATE);

    int current_id = 0;
    int current_file_id = 1;

    // 1. Create used blocks (mostly contiguous at first)
    for (int i = 0; i < used_blocks_count; ++i) {
        DiskBlock block = {current_id++, current_file_id, CONTIGUOUS};

        // Mark some blocks as fragmented
        if (i < fragmented_blocks_count) {
            block.type = FRAGMENTED;
        }

        // Change file ID every 10 blocks for simulation
        if (i % 10 == 0) {
            current_file_id++;
        }
        disk_array.push_back(block);
    }

    // 2. Create free blocks
    for (int i = used_blocks_count; i < TOTAL_BLOCKS; ++i) {
        disk_array.push_back({current_id++, -1, FREE});
    }

    // 3. Randomly shuffle the whole array to simulate fragmentation
    random_shuffle(disk_array.begin(), disk_array.end());

    // Reset IDs after shuffle for easier drawing
    for(int i = 0; i < TOTAL_BLOCKS; ++i) {
        disk_array[i].id = i;
    }
}

/**
 * @brief Draws the ASCII representation of the disk state.
 * @param disk_array The disk data.
 * @param title The title/status to display.
 */
void draw_disk(const vector<DiskBlock>& disk_array, const string& title) {
    clear_screen();
    cout << "--- " << title << " ---\n\n";
    cout << "Legend: \033[31mF\033[0m=Fragmented | \033[32mC\033[0m=Contiguous | \033[37m.\033[0m=Free Space\n";
    cout << "Total Blocks: " << TOTAL_BLOCKS << " (" << (TOTAL_BLOCKS * BLOCK_SIZE / 1024.0) << " GB Sim.)\n";
    cout << "Disk Map:\n";

    // Draw the grid
    for (int i = 0; i < TOTAL_BLOCKS; ++i) {
        const DiskBlock& block = disk_array[i];

        char symbol = '.';
        string color_code = "\033[37m"; // White (Free)

        if (block.type == FRAGMENTED) {
            symbol = 'F';
            color_code = "\033[31m"; // Red
        } else if (block.type == CONTIGUOUS) {
            symbol = 'C';
            color_code = "\033[32m"; // Green
        }

        cout << color_code << symbol << "\033[0m"; // Print colored symbol

        if ((i + 1) % DISK_WIDTH == 0) {
            cout << "\n";
        }
    }
    cout << "\n";
}

/**
 * @brief Simulates the Analysis stage.
 * @param disk_array The disk data.
 * @return Fragmentation percentage.
 */
float analyze_disk(const vector<DiskBlock>& disk_array) {
    int fragmented_count = 0;
    int used_count = 0;

    for (const auto& block : disk_array) {
        if (block.type != FREE) {
            used_count++;
            if (block.type == FRAGMENTED) {
                fragmented_count++;
            }
        }
    }

    if (used_count == 0) return 0.0f;
    return (static_cast<float>(fragmented_count) / used_count) * 100.0f;
}

/**
 * @brief Simulates the Optimization (Defragmentation) stage.
 * @param disk_array Reference to the disk data for modification.
 */
void optimize_disk(vector<DiskBlock>& disk_array) {
    int free_cursor = 0; // The target position for data
    int blocks_moved = 0;
    const int SLEEP_TIME = 10; // Animation speed

    // Find the first data block (where contiguous data should start)
    while (free_cursor < TOTAL_BLOCKS && disk_array[free_cursor].type != FREE) {
        free_cursor++;
    }

    if (free_cursor >= TOTAL_BLOCKS) {
        draw_disk(disk_array, "Optimization Skipped: Disk is already full and optimized.");
        sleep_ms(2000);
        return;
    }

    // Iterate through the entire disk starting from the current free_cursor
    for (int i = free_cursor; i < TOTAL_BLOCKS; ++i) {

        // We are looking for any data block (fragmented or contiguous) that is currently scattered.
        if (disk_array[i].type != FREE) {

            // If the block is already correctly positioned (i.e., before the free cursor), skip.
            // Since we start from the first free block (free_cursor), any data we find now is
            // a candidate for moving to consolidate free space.

            // 1. Temporarily store the data block
            DiskBlock data_block = disk_array[i];

            // 2. Move data_block to the free slot (free_cursor)
            disk_array[free_cursor].file_id = data_block.file_id;
            disk_array[free_cursor].type = CONTIGUOUS;

            // 3. Mark the block at index 'i' (the original location) as free.
            disk_array[i].file_id = -1;
            disk_array[i].type = FREE;

            // Advance the free cursor to the next expected free spot
            free_cursor++;

            blocks_moved++;

            // Visualization update (slow down for animation)
            if (blocks_moved % 5 == 0) { // Update every 5 moves
                draw_disk(disk_array, "Optimizing... Blocks Moved: " + to_string(blocks_moved));
                sleep_ms(SLEEP_TIME);
            }
        }
    }

    // Final redraw
    draw_disk(disk_array, "Optimization Complete! Blocks moved: " + to_string(blocks_moved));
    sleep_ms(2000);
}


// --- Main Program Loop ---

int main() {
    vector<DiskBlock> disk_c;
    char choice;

    // Initialize the disk on startup
    initialize_disk(disk_c);

    do {
        draw_disk(disk_c, "C: Drive Status");
        float frag_percent = analyze_disk(disk_c);

        cout << "\n------------------------------------------------\n";
        cout << "Current Fragmentation: " << frag_percent << "%\n";
        cout << "------------------------------------------------\n";
        cout << "(A) Analyze | (O) Optimize | (R) Re-fragment | (Q) Quit\n";
        cout << "Enter command: ";

        if (!(cin >> choice)) {
            // Handle bad input
            cin.clear();
            cin.ignore(numeric_limits<streamsize>::max(), '\n');
            continue;
        }

        choice = toupper(choice);

        if (choice == 'A') {
            draw_disk(disk_c, "Analyzing Disk...");
            sleep_ms(1000);
            cout << "\n\nAnalysis finished. Fragmentation is: " << frag_percent << "%\n";
            sleep_ms(1500);

        } else if (choice == 'O') {
            // Only optimize if fragmentation is > 1%
            if (frag_percent > 1.0f) {
                optimize_disk(disk_c);
            } else {
                draw_disk(disk_c, "Optimization not necessary (<1% fragmentation).");
                sleep_ms(1500);
            }
        } else if (choice == 'R') {
            initialize_disk(disk_c);
            draw_disk(disk_c, "Disk Re-Fragmented (Simulated Heavy Usage)");
            sleep_ms(1500);
        } else if (choice != 'Q') {
            cout << "Invalid choice. Please try again.\n";
            sleep_ms(1000);
        }

    } while (choice != 'Q');

    clear_screen();
    cout << "Drive Optimizer Simulator Exited.\n";
    return 0;
}

Overwriting drive_optimizer.cpp


### **Colab Cell 2: Compile**

This cell uses the g++ compiler to create an executable file named `drive_optimizer`.

In [10]:
!g++ drive_optimizer.cpp -o drive_optimizer -std=c++17

### **Colab Cell 3: Execute**

This cell runs the compiled program. You will interact with the simulator directly in the output area below this cell.

In [ ]:
!./drive_optimizer

--- C: Drive Status ---

Legend: F=Fragmented | C=Contiguous | .=Free Space
Total Blocks: 2500 (9.76562 GB Sim.)
Disk Map:
C.F..C.FCC.FCF..CF..FC.CFCF.CF.C.FCF.F..CCCCCCCF.C
.....CCFFC.CCF..FC.FF.CC..FC.CCFCC.FC.CCCCCCCCF.FF
CC.CCC.C.F.FC.CC.FCCF..FC.F...FCCCCCFC.C...C..F.CC
F.CCFFF..C.F.CC.CCF..FC..F...CFFCCCCC..F..CFF.FFFC
.CFF.FFCC.C.CCFFCC.C.CFF.FFCC.CCFFCC.CFCFFC.C.CF.C
CFC..FC.FCF.F...FCF.FCFCC.F.CC.CFFC...FCCC.CC.CF..
.CCFCCCCFF..FCC.CFF.FCFCC..C.F.C.FF..FFFC...FFCCCC
.CC.FFCC..CCC.FCFCCCCC..FC.FCCFFCC.CCCCCC.F...FC..
C..FF......CCCC....FCFF..C.CCF.CCCC..C.CFCC.CC.CF.
CC.FCCFC.FFCC..CCCC..CCF.CF..F.FFCF.FC.CC.....CC.F
CFC.CCC.F.CC..FFCF..FF..CC.CCFF.FCF..FF.FFCF..CC.C
CCFF..C.C..CFFCFC.FFFC.FCCF.CC.CFC.FCCF.CF.C.FFC..
CFFF..CC.CCC.CFF..C..CC..F..CCCFCCFFCC..C.CF.FFCC.
F.FCC...FFCF.F.FFFCCCF..FFCC.CCCF...C.CF.CFFCCCFCC
CCFCCFC..CCCF.FCCFC.CC.C.FC..CCF.CF.F.CCCFC...FCCC
.CCCF.CF.FFFF.C.CCFC.C..CC...CC.C.CFC...CF.C.F.F.C
C.CFC.CFCCCCC..CF...FCCCC..CCCFFCFC..F.CCCFCCFCF.C
CFCFCCF.CC